In [3]:
import bacteriopop_utils
import feature_selection_utils
import load_data
import dynamic_mode_decomposition as dmd
import network_construction as net

columns after aggregating on phylo level: Index([u'oxygen', u'replicate', u'week', u'kingdom', u'phylum', u'class',
       u'order', u'abundance'],
      dtype='object')
first (up to) 5 phylo columns to keep: ['' 'Acidilobales' 'Desulfurococcales' 'Sulfolobales' 'Thermoproteales']
('High', 1)
('High', 2)
('High', 3)
('High', 4)
('Low', 1)
('Low', 2)
('Low', 3)
('Low', 4)
dictionary keys: [('High', 4), ('Low', 1), ('High', 3), ('Low', 2), ('High', 2), ('Low', 3), ('High', 1), ('Low', 4)]
dataframe_dict.keys(): [('High', 4), ('Low', 1), ('High', 3), ('Low', 2), ('High', 2), ('Low', 3), ('High', 1), ('Low', 4)]
week                                                      4         5   \
phylo_concat                                                             
Archaea,,,                                          0.000000  0.000018   
Archaea,Crenarchaeota,,                             0.000000  0.000008   
Archaea,Crenarchaeota,Thermoprotei,                 0.000000  0.000007   
Archaea,Crenar

In [2]:
loaded_data = data = load_data.load_data()

In [3]:
loaded_data.shape

(64755, 11)

In [4]:
loaded_data.head()

,kingdom,phylum,class,order,family,genus,length,oxygen,replicate,week,abundance
sampleID,,,,,,,,,,,
1056013,Bacteria,Proteobacteria,Gammaproteobacteria,Methylococcales,Methylococcaceae,Methylobacter,9948861,Low,1,4,0.228531
1056013,Bacteria,Proteobacteria,Betaproteobacteria,Methylophilales,Methylophilaceae,Methylotenera,5066955,Low,1,4,0.220860
1056013,Bacteria,Bacteroidetes,Flavobacteriia,Flavobacteriales,Flavobacteriaceae,Flavobacterium,4654774,Low,1,4,0.054719
1056013,Bacteria,Proteobacteria,Gammaproteobacteria,Methylococcales,Methylococcaceae,,3046340,Low,1,4,0.047956
1056013,Bacteria,Proteobacteria,Gammaproteobacteria,,,,5620690,Low,1,4,0.040903


In [5]:
extracted_features = bacteriopop_utils.extract_features(
    dataframe = loaded_data,
    column_list = ['kingdom', 'phylum', 'class', 'order', 'family', 'genus', 'oxygen', 'abundance']
    # default list was: ['kingdom', 'phylum', 'class', 'order', 'family', 'genus', 'length', 'abundance', 'project']
    )

In [6]:
extracted_features.head()

,abundance,class=,class=Acidobacteriia,class=Actinobacteria,class=Alphaproteobacteria,class=Anaerolineae,class=Aquificae,class=Archaeoglobi,class=Bacilli,class=Bacteroidia,...,phylum=Planctomycetes,phylum=Poribacteria,phylum=Proteobacteria,phylum=Spirochaetes,phylum=Synergistetes,phylum=Tenericutes,phylum=Thaumarchaeota,phylum=Thermodesulfobacteria,phylum=Thermotogae,phylum=Verrucomicrobia
0,0.228531,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0.220860,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,0.054719,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.047956,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,0.040903,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [7]:
extracted_features.shape

(64755, 1450)

Just do PCA on a tiny bit of the data as a demo

In [8]:
pca_results = feature_selection_utils.pca_bacteria(
    data = extracted_features.head(100), n_components = 10)

In [ ]:
pca_results.components_

array([[-0.00272314, -0.03975207, -0.00449753, ...,  0.        ,
         0.        ,  0.        ],
       [-0.00259397,  0.19156772,  0.01794534, ...,  0.        ,
         0.        ,  0.        ],
       [-0.00975221, -0.07112615,  0.00182794, ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [ 0.02491242, -0.00092397,  0.00278457, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.00165731, -0.10002272,  0.03476809, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.00781743,  0.30522996, -0.02791468, ...,  0.        ,
         0.        ,  0.        ]])

Do correlations for a tiny subset of the data.

In [ ]:
feature_selection_utils.calculate_features_target_correlation(
    data = extracted_features.head(100),
    features = extracted_features.columns.tolist(),
    target='abundance',
    method="Pearson")

# Find the interaction network among bactria

In [5]:
mappings, nodes_list = dmd.find_fixed_adjacency_matrix(0,'order',False)

columns after aggregating on phylo level: Index([u'oxygen', u'replicate', u'week', u'kingdom', u'phylum', u'class',
       u'order', u'abundance'],
      dtype='object')
first (up to) 5 phylo columns to keep: ['' 'Acidilobales' 'Desulfurococcales' 'Sulfolobales' 'Thermoproteales']
('High', 1)
('High', 2)
('High', 3)
('High', 4)
('Low', 1)
('Low', 2)
('Low', 3)
('Low', 4)
dictionary keys: [('High', 4), ('Low', 1), ('High', 3), ('Low', 2), ('High', 2), ('Low', 3), ('High', 1), ('Low', 4)]
dataframe_dict.keys(): [('High', 4), ('Low', 1), ('High', 3), ('Low', 2), ('High', 2), ('Low', 3), ('High', 1), ('Low', 4)]
week                                                      4         5   \
phylo_concat                                                             
Archaea,,,                                          0.000000  0.000018   
Archaea,Crenarchaeota,,                             0.000000  0.000008   
Archaea,Crenarchaeota,Thermoprotei,                 0.000000  0.000007   
Archaea,Crenar

# Grpahs Visualization

In [6]:
for key in mappings.keys():
    Adj = mappings[key]
    nodes = nodes_list[key]
#     g = net.create_one_graph(Adj,nodes,edge_treshhold=1e-10)

## Reduce the adjacency matrix: 

In [20]:
mappings, nodes_list = dmd.find_fixed_adjacency_matrix(0,'order',False)
for key in mappings.keys():
    Adj,nodes=net.reduce_adjacency_matrix(mappings[key],nodes_list[key],edge_treshhold=0.5)
    file_name=str(key[0])+str(key[1])+'.pdf'
    net.adjacency_matrix_heatmap(Adj,nodes,str(key),file_name)

columns after aggregating on phylo level: Index([u'oxygen', u'replicate', u'week', u'kingdom', u'phylum', u'class',
       u'order', u'abundance'],
      dtype='object')
first (up to) 5 phylo columns to keep: ['' 'Acidilobales' 'Desulfurococcales' 'Sulfolobales' 'Thermoproteales']
('High', 1)
('High', 2)
('High', 3)
('High', 4)
('Low', 1)
('Low', 2)
('Low', 3)
('Low', 4)
dictionary keys: [('High', 4), ('Low', 1), ('High', 3), ('Low', 2), ('High', 2), ('Low', 3), ('High', 1), ('Low', 4)]
dataframe_dict.keys(): [('High', 4), ('Low', 1), ('High', 3), ('Low', 2), ('High', 2), ('Low', 3), ('High', 1), ('Low', 4)]
week                                                      4         5   \
phylo_concat                                                             
Archaea,,,                                          0.000000  0.000018   
Archaea,Crenarchaeota,,                             0.000000  0.000008   
Archaea,Crenarchaeota,Thermoprotei,                 0.000000  0.000007   
Archaea,Crenar

AttributeError: 'module' object has no attribute 'reduce_adjacency_matrix'